<a href="https://colab.research.google.com/github/Miseq/dw_matrix_road_sign_detection/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import os
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import color, exposure 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import  to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign_detection/data'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign_detection/data


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')
x_train, y_train = train['features'], train['labels']
x_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape = x_train.shape[1:]
num_clases = y_train.shape[1]


In [0]:
def train_model(model, x_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      x_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (x_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

In [0]:
def predict(model_trained, x_test, y_test, scoring=accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(x_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, x_train, y_train)
  return predict(model_trained, x_test, y_test)

In [0]:
def get_cnn_v3(input_shape,num_clases):
  return Sequential([
                    
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        
                    MaxPool2D(),
                    Dropout(0.3),
                    
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

                     MaxPool2D(),
                    Dropout(0.3),
                   
                   Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

                     MaxPool2D(),
                    Dropout(0.3),

                    Flatten(),
                    Dense(1024, activation='relu'),
                    Dropout(0.3),
                    Dense(num_clases, activation='softmax'),
]) 


In [10]:
model = get_cnn_v3(input_shape, num_clases)  # uczymy na szarych, shpae: 32, 32,1
model_trained = train_model(model, x_train, y_train, params_fit={})
predict(model_trained, x_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.5512 - accuracy: 0.3294 - val_loss: 0.7875 - val_accuracy: 0.7795
Epoch 2/5
272/272 [==============================] - 3s 13ms/step - loss: 0.7734 - accuracy: 0.7615 - val_loss: 0.2010 - val_accuracy: 0.9532
Epoch 3/5
272/272 [==============================] - 3s 13ms/step - loss: 0.3631 - accuracy: 0.8913 - val_loss: 0.0738 - val_accuracy: 0.9799
Epoch 4/5
272/272 [==============================] - 3s 12ms/step - loss: 0.2306 - accuracy: 0.9330 - val_loss: 0.0444 - val_accuracy: 0.9878
Epoch 5/5
272/272 [==============================] - 3s 13ms/step - loss: 0.1715 - accuracy: 0.9504 - val_loss: 0.0369 - val_accuracy: 0.9909


0.963718820861678

In [11]:
model_trained.evaluate(x_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1319 - accuracy: 0.9637


0.963718831539154

In [0]:
def get_model(params):
  return Sequential([
                    
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        
                    MaxPool2D(),
                    Dropout(params['dopout_cnn_block_one']),
                    
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

                     MaxPool2D(),
                    Dropout(params['dopout_cnn_block_two']),
                   
                   Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

                     MaxPool2D(),
                    Dropout(params['dopout_cnn_block_three']),

                    Flatten(),
                    Dense(1024, activation='relu'),
                    Dropout(params['dopout_dense_one']),
                    
                    Dense(num_clases, activation='relu'),
                    Dropout(params['dopout_dense_two']),
  
                    Dense(num_clases, activation='softmax')                   
]) 
  


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      x_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
      )

  score = model.evaluate(x_test, y_test, verbose=0)
  acc = score[1]
  print(params, 'accuracy={}'.format(acc))

  return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [29]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dopout_cnn_block_one': hp.uniform('dopout_cnn_block_one', 0.3, 0.5),
    'dopout_cnn_block_two': hp.uniform('dopout_cnn_block_two', 0.5, 0.5),
    'dopout_cnn_block_three': hp.uniform('dopout_cnn_block_three', 0.3, 0.5),

    'dopout_dense_one': hp.uniform('dopout_dense_one', 0.3, 0.7),
    'dopout_dense_two': hp.uniform('dopout_dense_two', 0.3 , 0.7)

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    20,
    Trials()
)

{'batch_size': 200.0, 'dopout_cnn_block_one': 0.4752265452567715, 'dopout_cnn_block_three': 0.3413790038635084, 'dopout_cnn_block_two': 0.5, 'dopout_dense_one': 0.5230556448977967, 'dopout_dense_two': 0.4593705127514261}
accuracy=0.46371883153915405
{'batch_size': 140.0, 'dopout_cnn_block_one': 0.4096516511240827, 'dopout_cnn_block_three': 0.4416041627746028, 'dopout_cnn_block_two': 0.5, 'dopout_dense_one': 0.42050611582359854, 'dopout_dense_two': 0.5208788105197198}
accuracy=0.054421767592430115
{'batch_size': 200.0, 'dopout_cnn_block_one': 0.44436446220262305, 'dopout_cnn_block_three': 0.36382452503633517, 'dopout_cnn_block_two': 0.5, 'dopout_dense_one': 0.6544628831531029, 'dopout_dense_two': 0.40975488761032985}
accuracy=0.680272102355957
{'batch_size': 130.0, 'dopout_cnn_block_one': 0.44526007610767515, 'dopout_cnn_block_three': 0.49136304779047746, 'dopout_cnn_block_two': 0.5, 'dopout_dense_one': 0.49795082370851895, 'dopout_dense_two': 0.3949618567689118}
accuracy=0.405668944120